# Hello World Pipeline

## Section 1: Introduction

The goal of this notebook is to show how you can turn a notebook into an API.

## Section 2: Pipeline API

In [ ]:
# pipeline-api
message = 'hello world'

In [ ]:
# pipeline-api
def pipeline_api(
    file,
    file_content_type=None,
    m_some_parameters=[],
):
    from layoutparser.models import Detectron2LayoutModel
    from PIL import Image
    from pdf2image import convert_from_bytes
    model = Detectron2LayoutModel(
        config_path="lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config",  # In model catalog
        label_map={
            0: "Text",
            1: "Title",
            2: "List",
            3: "Table",
            4: "Figure",
        },  # In model`label_map`
        extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],  # Optional
    )

    if file_content_type=='image/png':
        img = Image.open(file)
        detections = model.detect(img)
    if file_content_type=='application/pdf':
        pages = convert_from_bytes(file.read(), 500)
        results= []
        for page in pages:
            detections = model.detect(page)
            results.append(detections)
        return results